In [8]:
import pandas as pd
from collections import defaultdict
import plotly.express as px
import ast
import re

In [9]:
df_teps = pd.read_csv('tep_merged.csv', low_memory=False, sep=';')
df_permits = pd.read_csv('permit_page_merged.csv',
                         low_memory=False,
                         sep=';')

## Hypothesis

- Загальний аналіз

[H1] Реальна вартість будівництва об’єктів за державні кошти більша ніж за приватні.

[H2] У невеликих громадах будівництво аналогічних об'єктів обходиться дорожче в 2-3 рази.
.
[H4] Є об'єкти з підозріло коротким терміном реалізації (≤2 міс.) при високій вартості — можливі фіктивні проєкти.

[H5] Ціна за м² знижується або зростає з більшою кількістю поверхів (аналіз економії масштабу).

[H6] Об'єкти, що будуються “господарським способом”, мають системно нижчу вартість, ніж підрядні.

[H7] У Києві/обласних центрах — вища ціна за м², ніж у райцентрах або сільській місцевості.

[H8] Будівництво у прикордонних регіонах (Херсон, Суми, Чернігів) демонструє аномалії у вартості та темпах

[H10] Кількість поверхів ≠ 1, але “загальна площа” дуже мала — можливе дроблення або помилки.

[H11] Для об'єктів з ДКБС-кодом, що мають виробниче або енергетичне призначення — середня вартість суттєво відрізняється.

[H12] Торгові приміщення (магазини) мають вартість м² близьку до житлових, але з коротшими термінами будівництва.

[H13] Підозрілі об'єкти мають: високу вартість, короткий термін, багато “нулів” у ТЕП, нульову корисну площу або 0 поверхів.

[H14] Найбільш аномальні об'єкти зустрічаються серед приватних замовників, а не держорганів.


## Cleaning tep

In [10]:
# Calculate non-null percentage for each column
percentages = {
    col: round(df_teps[col].notna().sum() / df_teps.shape[0] * 100, 2)
    for col in df_teps.columns
}
# Convert to DataFrame and sort
percent_df = (
    pd.DataFrame(percentages.items(), columns=["Column", "Non-Null %"])
    .sort_values(by="Non-Null %", ascending=False)
    .reset_index(drop=True)
)
percent_df.head(35)

,Column,Non-Null %
0,Назва об'єкта,100.00
1,Тип об'єкта,100.00
2,doc_id,100.00
3,Вид будівництва,99.99
4,"Загальна площа приміщень, м2",92.95
5,"Площа забудови, м2",88.16
6,"Кількість поверхів, од",86.93
7,"Загальна площа будівлі, м2",86.92
8,Код ДКБС,80.95
9,"Житлова площа приміщень, м2",67.92


In [11]:
# Ключові слова для групування
keywords = [
    "площа", "вартість", "поверх", "обʼєм", "об'єм", "висота", 
    "кошторис", "вбудован", "забудов", "будівельн", "житлова", 
    "нежитлова", "квартира", "приміщен", "будинок", "корисна", 
    "озеленення", "даху", "цоколь", "технічна", "тротуар"
]

# Припустимо, що у тебе вже є DataFrame `percent_df` з колонкою "Column"
percent_df["Column_lower"] = percent_df["Column"].str.lower()

# Групування колонок за ключовими словами
grouped_columns = defaultdict(list)
for kw in keywords:
    mask = percent_df["Column_lower"].str.contains(kw)
    matches = percent_df[mask]["Column"].tolist()
    if matches:
        grouped_columns[kw].extend(matches)

# Перетворення словника на DataFrame для зручного перегляду
grouped_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in grouped_columns.items()]))

# Вивід результату
grouped_df.head(2)


,площа,вартість,поверх,об'єм,висота,кошторис,вбудован,забудов,будівельн,житлова,нежитлова,приміщен,корисна,озеленення,даху,цоколь,тротуар
0,"Загальна площа приміщень, м2","Кошторисна вартість,, тис. грн","Кількість поверхів, од","Загальний будівельний об'єм, м3","Гранична висота будівлі/споруди, м","Кошторисна вартість,, тис. грн",Площа вбудованих нежитлових приміщень (вбудова...,"Площа забудови, м2","Загальний будівельний об'єм, м3","Житлова площа приміщень, м2","Нежитлова площа, м2","Загальна площа приміщень, м2","Корисна площа, м2","Площа озеленення, м2","Площа покриття даху, м2","Цокольний поверх, од","Площа тротуарів, м2"
1,"Площа забудови, м2","Кошторисна вартість,, тис. грн (примітка)","Кількість поверхів, од (примітка)","Будівельний об'єм вище відм. 0.00, м3","Висота будівлі, м","Кошторисна вартість,, тис. грн (примітка)",Площа вбудованих нежитлових приміщень (вбудова...,"Площа забудови, м2 (примітка)","Будівельний об`єм, м3","Нежитлова площа, м2","Нежитлова площа, м2 (примітка)","Житлова площа приміщень, м2","Корисна площа, м2 (примітка)",Загальна площа під зеленими насадженнями (озел...,"Площа покриття даху, м2 (примітка)","Цокольний поверх, од (примітка)","Площа тротуару, м2"


ШІ допоміг обробити ці дані

In [12]:
# Площа об'єкта
total_area_cols = [
    'Загальна площа приміщень, м2',
    'Загальна площа будівлі, м2',
    'Загальна площа, м2',
    'Площа будинку, м2',
    'Площа житлового будинку, м2',
    'Площа квартир у будинку, м2'
]

# Житлова площа
living_area_cols = [
    'Житлова площа приміщень, м2',
    'Житлова площа однокімнатних квартир, м2',
    'Житлова площа двокімнатних квартир, м2',
    'Житлова площа трикімнатних квартир, м2',
    'Житлова площа чотирикімнатних квартир, м2',
    'Житлова площа п’ятикімнатних квартир, м2',
    'Житлова площа шестикімнатнатних квартир, м2',
    'Житлова площа семикімнатних квартир, м2',
    'Житлова площа восьмикімнатних і більше квартир, м2'
]

# Нежитлова площа
nonresidential_area_cols = [
    'Нежитлова площа, м2',
    'Площа допоміжна нежитлова, м2',
    'Площа інших нежитлових приміщень, м2'
]

# Площа забудови
footprint_cols = [
    'Площа забудови, м2',
    'Площа основи (забудови), м2',
    'Площа забудови , га'
]

# Кількість поверхів
floor_cols = [
    'Кількість поверхів, од',
    'Поверховість, поверхів',
    'Кількість надземних поверхів, од',
]

# Обʼєм будівлі
volume_cols = [
    'Загальний будівельний об\'єм, м3',
    'Об\'єм будівлі, м3',
    'Будівельний об\'єм вище відм. 0.00, м3',
    'Будівельний об\'єм нижче відм. 0.00, м3'
]

# Висота
height_cols = [
    'Гранична висота будівлі/споруди, м',
    'Висота будівлі, м',
    'Висота, м'
]

# Вартість
cost_cols = [
    'Кошторисна вартість,, тис. грн',
    'Вартість, тис. грн',
    'Вартість основних фондів об’єкту будівництва, тис. грн'
]


Аналізую дані по колонках

In [13]:
df_teps[total_area_cols].sample(10)

,"Загальна площа приміщень, м2","Загальна площа будівлі, м2","Загальна площа, м2","Площа будинку, м2","Площа житлового будинку, м2","Площа квартир у будинку, м2"
29465,118.0,118.0,NaN,NaN,NaN,NaN
33194,295.6,295.6,NaN,NaN,NaN,NaN
10142,242.0,242.0,NaN,NaN,NaN,NaN
30128,18.1,18.1,NaN,NaN,NaN,NaN
18482,61.5,61.5,NaN,NaN,NaN,NaN
32443,30.7,NaN,36.3,NaN,NaN,NaN
18781,168.7,168.7,NaN,NaN,NaN,NaN
35315,178.4,178.4,NaN,NaN,NaN,NaN
19978,58.3,58.3,58.3,NaN,58.3,NaN
22697,118.9,118.9,NaN,NaN,NaN,NaN


Створюю нові колонки для датафрейму та зменшую його

In [14]:
# Для створення агрегованих колонок
def first_valid(df, columns):
    return df[columns].bfill(axis=1).infer_objects(copy=False).iloc[:, 0]

# Створення нових колонок
df_teps['total_area_m2'] = first_valid(df_teps, total_area_cols)
df_teps['living_area_m2'] = first_valid(df_teps, living_area_cols)
df_teps['nonresidential_area_m2'] = first_valid(df_teps, nonresidential_area_cols)
df_teps['build_footprint_area_m2'] = first_valid(df_teps, footprint_cols)
df_teps['total_floors'] = first_valid(df_teps, floor_cols)
df_teps['building_volume_m3'] = first_valid(df_teps, volume_cols)
df_teps['building_height_m'] = first_valid(df_teps, height_cols)
df_teps['estimated_cost_uah'] = first_valid(df_teps, cost_cols)

# Колонки, які залишаємо як є (можеш доповнити)
keep_cols = [
    "doc_id",
    "Назва об'єкта",
    "Тип об'єкта",
    'Вид будівництва',
    "Кількість підземних поверхів, од",
    'Тривалість будівництва, міс',
    'Тривалість експлуатації (Розрахунковий строк експлуатації), р.',
    'Кількість житлових кімнат, од',
    "Код ДКБС"
]

# Остаточний датафрейм з очищеними колонками
selected_cols = [col for col in keep_cols if col in df_teps.columns] +\
[
    'total_area_m2',
    'living_area_m2',
    'nonresidential_area_m2',
    'build_footprint_area_m2',
    'total_floors',
    'building_volume_m3',
    'building_height_m',
    'estimated_cost_uah'
]

df_teps_cleaned = df_teps[selected_cols]


C:\Users\dimit\AppData\Local\Temp\ipykernel_29192\104804955.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df[columns].bfill(axis=1).infer_objects(copy=False).iloc[:, 0]
C:\Users\dimit\AppData\Local\Temp\ipykernel_29192\104804955.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return df[columns].bfill(axis=1).infer_objects(copy=False).iloc[:, 0]
C:\Users\dimit\AppData\Local\Temp\ipykernel_29192\104804955.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call 

In [15]:
teps_rename_map = {
    "Назва об'єкта": "object_name_tep",
    "Тип об'єкта": "object_type_tep",
    "Вид будівництва": "construction_type_tep",
    "Кількість підземних поверхів, од": "underground_floors",
    "Тривалість будівництва, міс": "construction_duration_months",
    "Тривалість експлуатації (Розрахунковий строк експлуатації), р.": "lifetime_years",
    "Кількість житлових кімнат, од": "living_rooms_count",
    "Код ДКБС": "dkbs_code"
}

# Перейменовуємо
df_teps_cleaned = df_teps_cleaned.rename(columns=teps_rename_map)


In [16]:
df_teps_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35835 entries, 0 to 35834
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   doc_id                        35835 non-null  object 
 1   object_name_tep               35834 non-null  object 
 2   object_type_tep               35835 non-null  object 
 3   construction_type_tep         35832 non-null  object 
 4   underground_floors            5732 non-null   float64
 5   construction_duration_months  6099 non-null   object 
 6   lifetime_years                4936 non-null   object 
 7   living_rooms_count            21229 non-null  float64
 8   dkbs_code                     29010 non-null  object 
 9   total_area_m2                 34932 non-null  object 
 10  living_area_m2                24376 non-null  object 
 11  nonresidential_area_m2        20794 non-null  float64
 12  build_footprint_area_m2       31909 non-null  object 
 13  t

## Cleaning permit documents

In [17]:
def extract_first_tep(x):
    try:
        parsed = ast.literal_eval(x)
        if isinstance(parsed, list) and len(parsed) > 0:
            return parsed[0]
    except (ValueError, SyntaxError):
        pass
    return None

df_permits.loc[:, 'first_tep'] = df_permits['teps'].apply(extract_first_tep)

In [18]:
cost_cols = [
    "Загальна, тис. грн._Кошторисна вартість будівництва",
    "За проектом, тис. грн._Кошторисна вартість будівництва"
]

df_permits['cost'] = first_valid(df_permits, cost_cols)

In [19]:
columns_to_extract = [
    "Реєстраційний номер в ЄДЕССБ_Основна інформація",
    "Тип документу_Основна інформація",
    "Статус документу_Основна інформація",
    "Тип_Основна інформація",
    "Версія документу_Основна інформація",
    "Статус реєстрації_Основна інформація",
    "Орган, що видав_Основна інформація",
    "Назва об’єкта_Основна інформація",
    "Форма подачі документа_Основна інформація",
    "Клас наслідків_Основна інформація",
    "Адреса_Місце розташування об'єкта будівництва та адреса",
    "Дата реєстрації в ЄДЕССБ_Основна інформація",
    "Вид будівництва_Відомості про об'єкт",
    "Клас наслідків_Відомості про об'єкт",
    "Спосіб будівництва_Відомості про об'єкт",
    "Тип_Відомості про об'єкт",
    "cost",
    "Вид будівництва_Будівельний паспорт",
    "first_tep"
]

In [20]:
# Calculate non-null percentage for each column
percentages = {
    col: round(df_permits[col].notna().sum() / df_permits.shape[0] * 100, 2)
    for col in columns_to_extract
}
# Convert to DataFrame and sort
percent_df = (
    pd.DataFrame(percentages.items(), columns=["Column", "Non-Null %"])
    .sort_values(by="Non-Null %", ascending=False)
    .reset_index(drop=True)
)
percent_df.head(40)


,Column,Non-Null %
0,Реєстраційний номер в ЄДЕССБ_Основна інформація,100.00
1,Тип документу_Основна інформація,100.00
2,Статус документу_Основна інформація,100.00
3,Тип_Основна інформація,100.00
4,Версія документу_Основна інформація,100.00
5,Статус реєстрації_Основна інформація,100.00
6,"Орган, що видав_Основна інформація",100.00
7,Назва об’єкта_Основна інформація,100.00
8,Форма подачі документа_Основна інформація,100.00
9,Клас наслідків_Основна інформація,100.00


In [21]:
df_selected = df_permits[
    [
        "Реєстраційний номер в ЄДЕССБ_Основна інформація",
        "Тип документу_Основна інформація",
        "Статус документу_Основна інформація",
        "Тип_Основна інформація",
        "Версія документу_Основна інформація",
        "Статус реєстрації_Основна інформація",
        "Орган, що видав_Основна інформація",
        "Назва об’єкта_Основна інформація",
        "Форма подачі документа_Основна інформація",
        "Клас наслідків_Основна інформація",
        "Адреса_Місце розташування об'єкта будівництва та адреса",
        "Дата реєстрації в ЄДЕССБ_Основна інформація",
        "Вид будівництва_Відомості про об'єкт",
        "Клас наслідків_Відомості про об'єкт",
        "Спосіб будівництва_Відомості про об'єкт",
        "Тип_Відомості про об'єкт",
        "cost",
        "Вид будівництва_Будівельний паспорт",
        "Відомості про замовника_Назва",
        "first_tep"
    ]
].rename(columns={
    "Реєстраційний номер в ЄДЕССБ_Основна інформація": "registration_number_edessb",
    "Тип документу_Основна інформація": "document_type",
    "Статус документу_Основна інформація": "document_status",
    "Тип_Основна інформація": "main_type",
    "Версія документу_Основна інформація": "document_version",
    "Статус реєстрації_Основна інформація": "registration_status",
    "Орган, що видав_Основна інформація": "issuing_body",
    "Назва об’єкта_Основна інформація": "object_name",
    "Форма подачі документа_Основна інформація": "submission_format",
    "Клас наслідків_Основна інформація": "impact_class_main",
    "Адреса_Місце розташування об'єкта будівництва та адреса": "address",
    "Дата реєстрації в ЄДЕССБ_Основна інформація": "registration_date",
    "Вид будівництва_Відомості про об'єкт": "construction_type",
    "Клас наслідків_Відомості про об'єкт": "impact_class",
    "Спосіб будівництва_Відомості про об'єкт": "construction_method",
    "Тип_Відомості про об'єкт": "object_type",
    "Вид будівництва_Будівельний паспорт": "construction_type_passport",
    "Відомості про замовника_Назва": "client_name"
})


## Analysis with cost and without

In [22]:
# 1. Розбивка на наявні та відсутні за кошторисною вартістю
has_cost = df_selected[~df_selected["cost"].isna()]
no_cost = df_selected[df_selected["cost"].isna()]

# 2. Функція для порівняльного аналізу
def compare_distribution(col):
    return (
        pd.DataFrame({
            'with_cost%': has_cost[col].value_counts(normalize=True).mul(100).round(2),
            'without_cost%': no_cost[col].value_counts(normalize=True).mul(100).round(2),
            'count_with_cost': has_cost[col].value_counts(),
            'count_without_cost': no_cost[col].value_counts()
        })
        .fillna(0)
        .sort_values("with_cost%", ascending=False)
    )


# 4. Порівняння для main_type
dist_main_type = compare_distribution("main_type")

# Порівняння для document_status
dist_doc_status = compare_distribution("document_status")

# Порівняння для submission_format
dist_submission_format = compare_distribution("submission_format")

# Порівняння для object_type
dist_object_type = compare_distribution("object_type")


In [23]:
dist_object_type

,with_cost%,without_cost%,count_with_cost,count_without_cost
object_type,,,,
Будівельний паспорт,62.03,58.55,26192,27457
Відсутні відомості про містобудівні умови та обмеження,17.03,0.60,7189,283
Містобудівні умови та обмеження,14.40,9.59,6080,4497
Лист відсутній,5.84,3.80,2467,1784
Містобудівні умови та обмеження не надаються,0.64,0.28,271,130
Схема забудови земельної ділянки,0.04,26.34,16,12352
Відомості про будівельний паспорт відсутні,0.02,0.82,8,384
"Висновок про розміщення потужностей підприємст, об'єктів для проживання ВПО",0.00,0.01,1,3
Відомості з містобудівної документації,0.00,0.01,1,4


In [24]:
dist_submission_format

,with_cost%,without_cost%,count_with_cost,count_without_cost
submission_format,,,,
Паперова (ЦНАП),78.59,74.66,37378,63664
Електронна (розглянуто),21.40,25.33,10180,21601
Паперова,0.01,0.01,4,8
Електронна,0.00,0.00,1,1


In [25]:
dist_doc_status

,with_cost%,without_cost%,count_with_cost,count_without_cost
document_status,,,,
Діючий,78.33,79.41,37255.0,67716
Повернуто на доопрацювання,18.46,16.12,8779.0,13749
Повернуто без розгляду,2.49,3.14,1185.0,2680
Відмовлено,0.65,1.21,308.0,1033
Скасований,0.07,0.08,35.0,64
Відмовлено у внесенні відомостей в систему,0.00,0.04,1.0,31
Зупинений,0.00,0.00,0.0,1


In [26]:
dist_main_type

,with_cost%,without_cost%,count_with_cost,count_without_cost
main_type,,,,
Реєстрація декларації про готовність до експлуатації об’єкта,80.07,3.36,38082.0,2867
Повідомлення про початок виконання будівельних робіт,17.24,49.43,8199.0,42154
Видача сертифіката про прийняття в експлуатацію закінчених будівництвом об’єктів,2.63,0.84,1252.0,713
"Реєстрація декларації про готовність до експлуатації об’єкта, за рішенням суду",0.06,0.33,30.0,284
Видача дозволу на виконання будівельних робіт,0.00,2.91,0.0,2479
Повідомлення про початок виконання підготовчих робіт,0.00,0.03,0.0,23
Реєстрація декларації про готовність до експлуатації об’єкта за амністією,0.00,43.10,0.0,36754


In [27]:

# 1. Перетворення дати
df_selected["registration_date"] = pd.to_datetime(df_selected["registration_date"], format="%d.%m.%Y", errors="coerce")

# 2. Позначка наявності вартості
df_selected["has_cost"] = df_selected["cost"].notna()

# 3. Групування по місяцях
df_selected["week"] = df_selected["registration_date"].dt.to_period("W").dt.to_timestamp()

# 4. Підрахунок
weekly_counts = (
    df_selected
    .groupby(["week", "has_cost"])
    .size()
    .reset_index(name="count")
    .replace({True: "З вартістю", False: "Без вартості"})
)

# 5. Побудова графіка
fig = px.line(
    weekly_counts,
    x="week",
    y="count",
    color="has_cost",
    labels={"week": "Дата", "count": "Кількість документів", "has_cost": "Наявність кошторису"},
    title="Динаміка дозвільних документів з/без кошторисної вартості"
)

fig.show()


різниця є у типах відомостей про об'єкта та типах Дозвільного документу

Динаміка дозвільни документів з/без кошторисної вартості однакова для обох випадків

## Mergind dataframes

In [28]:
# Перейменовуємо колонку у df_selected, щоб зручно зʼєднати
df_selected = df_selected.rename(columns={"first_tep": "doc_id"})

# З'єднуємо дозвільні дані до ТЕПів (тільки для наявних ТЕПів)
df_teps_enriched = df_teps_cleaned.merge(
    df_selected,
    how="left",
    on="doc_id"
)


In [29]:
# Calculate non-null percentage for each column
percentages = {
    col: round(df_teps_enriched[col].notna().sum() / df_teps_enriched.shape[0] * 100, 2)
    for col in df_teps_enriched.columns
}
# Convert to DataFrame and sort
percent_df = (
    pd.DataFrame(percentages.items(), columns=["Column", "Non-Null %"])
    .sort_values(by="Non-Null %", ascending=False)
    .reset_index(drop=True)
)
percent_df.head(60)


,Column,Non-Null %
0,doc_id,100.00
1,object_name_tep,100.00
2,object_type_tep,100.00
3,main_type,100.00
4,document_status,100.00
5,document_type,100.00
6,client_name,100.00
7,registration_date,100.00
8,address,100.00
9,document_version,100.00


In [30]:
percent_df.to_csv('df_teps_enriched_perc.csv')

In [31]:
df_teps_enriched.dropna(subset='cost', inplace=True)

витягуємо адресу

In [32]:
# Список міст з населенням понад 100 тис.
big_cities_raw = """
Київ, Харків, Дніпро, Одеса, Донецьк, Запоріжжя, Львів, Кривий Ріг, Миколаїв, Маріуполь,
Луганськ, Макіївка, Вінниця, Сімферополь, Севастополь, Херсон, Полтава, Чернігів, Черкаси,
Суми, Горлівка, Хмельницький, Житомир, Кам'янське, Кропивницький, Рівне, Чернівці, Кременчук,
Тернопіль, Івано-Франківськ, Луцьк, Біла Церква, Краматорськ, Мелітополь, Керч, Нікополь,
Слов'янськ, Бердянськ, Сіверськодонецьк, Алчевськ, Павлоград, Ужгород
"""

# Очистимо список і збережемо у CSV
big_cities = [city.strip() for city in big_cities_raw.split(",") if city.strip()]
df_big_cities = pd.DataFrame(big_cities, columns=["city"])

In [33]:
def split_address_parts(addr: str) -> dict:
    if pd.isna(addr):
        return {"region": None, "district": None, "settlement": None, "is_small": None, "is_big_city": None}

    # 1. Прибрати вставки у дужках
    addr = re.sub(r"\(.*?\)", "", addr).strip()

    # 2. Розбити
    parts = [p.strip() for p in addr.split(",") if p.strip()]
    region = next((p for p in parts if "обл." in p or "м. Київ" in p or "м. Севастополь" in p), None)
    district = next((p for p in parts if "район" in p), None)

    # 3. Населений пункт
    settlement = next((p for p in parts if re.search(r"\b[см]\. ", p)), None)
    if not settlement:
        # fallback: останній компонент
        settlement = parts[-1] if parts else None

    # 4. Витяг імені населеного пункту без префіксів ("с.", "м." тощо)
    core_name_match = re.search(r"(?:[см]\. )?([\wʼ’ІіЄєЇїҐґА-Яа-я\-ʼ ]+)", settlement or "")
    core_name = core_name_match.group(1).strip() if core_name_match else None

    # 5. Ознака "велике місто"
    is_big_city = core_name in big_cities

    # 6. Ознака "маленьке місто"
    is_small = any([
        "громада" in addr.lower(),
        "с." in addr.lower(),
        "смт" in addr.lower(),
        "ОТГ" in addr,
        "територіальна громада" in addr.lower()
    ]) and not is_big_city

    return {
        "region": region,
        "district": district,
        "settlement": core_name,
        "is_small": is_small,
        "is_big_city": is_big_city
    }


In [34]:
split_address_parts(df_teps_enriched['address'][33333])

{'region': 'Львівська обл.',
 'district': 'Шептицький район',
 'settlement': 'Бутини',
 'is_small': True,
 'is_big_city': False}

In [35]:
address_parts = df_teps_enriched["address"].apply(split_address_parts).apply(pd.Series)
df_teps_enriched = pd.concat([df_teps_enriched, address_parts], axis=1)

In [36]:
print(f"Big city documents {df_teps_enriched.is_big_city.sum()}")
print(f"Small city documents {df_teps_enriched.is_small.sum()}")

Big city documents 9825
Small city documents 32838


## EDA

In [42]:
df_teps_enriched['dkbs_code'].value_counts(dropna=False, normalize=False).head(5)

dkbs_code
1110.3 Будинки садибного типу                  22040
NaN                                             9749
1110.4 Будинки дачні та садові                  3659
1230.1 Торгові центри, універмаги, магазини     1158
1230.9 Будівлі торговельні інші                  752
Name: count, dtype: int64

In [38]:
df_teps_enriched['object_type_tep'].value_counts(dropna=False, normalize=True)

object_type_tep
Житловий будинок садибного типу                                        0.530557
Будівля                                                                0.187187
Садовий будинок                                                        0.083025
Приміщення (група приміщень)                                           0.046515
Квартира                                                               0.044242
Будинок                                                                0.034096
Група будівель/споруд                                                  0.019356
Лінійний об’єкт інженерно-транспортної інфраструктури                  0.017013
Споруда                                                                0.012115
Комплекс (будова)                                                      0.012068
Дачний будинок                                                         0.006866
Гаражний бокс                                                          0.002906
Група відокремлених прим

## Гіпотези

### [H1] Реальна вартість будівництва об’єктів за державні кошти більша ніж за приватні.

неможливо перевірити за чиї кошти будувався об'єкт

### [H2] У невеликих громадах будівництво аналогічних об'єктів обходиться дорожче в 2-3 рази.

In [39]:
# 1. Розрахунок вартості за м²
df_teps_enriched["cost_per_m2"] = pd.to_numeric(df_teps_enriched["cost"], errors="coerce") / pd.to_numeric(df_teps_enriched["total_area_m2"], errors="coerce")

# 2. Відбір валідних значень
df_valid = df_teps_enriched[
    (df_teps_enriched["cost_per_m2"] > 0) &
    (~df_teps_enriched["is_small"].isna())
]

# 3. Групування за типом обʼєкта та масштабом громади
summary = (
    df_valid
    .groupby(["dkbs_code", "is_small"])
    .agg(
        median_cost_per_m2=("cost_per_m2", "median"),
        count=("cost_per_m2", "count")
    )
    .unstack()
)

# 4. Додатково — коефіцієнт різниці (малі / великі)
summary[("cost_ratio", "small_vs_big")] = summary[("median_cost_per_m2", True)] / summary[("median_cost_per_m2", False)]

# Фільтруємо по count у малих або великих громадах
filtered = summary[
    (summary[("count", True)] > 100) | (summary[("count", False)] > 100)
]

filtered.sort_values(by=[("count", True), ("count", False)], ascending=False)

median_cost_per_m2  \
is_small                                                        False   
dkbs_code                                                               
1110.3 Будинки садибного типу                                5.729107   
1110.4 Будинки дачні та садові                               7.142857   
1230.1 Торгові центри, універмаги, магазини                  9.289279   
1230.9 Будівлі торговельні інші                              8.424600   
1242.1 Гаражі наземні                                        4.311581   
1122.1 Будинки багатоквартирні масової забудови              9.214659   
1263.3 Будівлі шкіл та інших середніх навчальни...           7.579660   
1252.8 Склади універсальні                                   5.859347   
1271.9 Будівлі сільськогосподарського призначен...           7.440660   
1230.3 Станції технічного обслуговування автомо...           3.532871   
1252.9 Склади та сховища інші                                8.425836   
1122.2 Будинки багатоквартирні підвищеної комфо...           9.647680   

                                                                count  \
is_small                                                 True   False   
dkbs_code                                                               
1110.3 Будинки садибного типу                        6.912281  2716.0   
1110.4 Будинки дачні та садові                       7.501225   378.0   
1230.1 Торгові центри, універмаги, магазини          8.649763   345.0   
1230.9 Будівлі торговельні інші                      7.607144   215.0   
1242.1 Гаражі наземні                                5.399543   128.0   
1122.1 Будинки багатоквартирні масової забудови      7.373176   226.0   
1263.3 Будівлі шкіл та інших середніх навчальни...  15.924208    90.0   
1252.8 Склади універсальні                           6.059286    84.0   
1271.9 Будівлі сільськогосподарського призначен...   4.571830     5.0   
1230.3 Станції технічного обслуговування автомо...   8.457924    82.0   
1252.9 Склади та сховища інші                        4.011281    52.0   
1122.2 Будинки багатоквартирні підвищеної комфо...   4.951395   111.0   

                                                              cost_ratio  
is_small                                               True small_vs_big  
dkbs_code                                                                 
1110.3 Будинки садибного типу                       19231.0     1.206520  
1110.4 Будинки дачні та садові                       3275.0     1.050172  
1230.1 Торгові центри, універмаги, магазини           808.0     0.931156  
1230.9 Будівлі торговельні інші                       534.0     0.902968  
1242.1 Гаражі наземні                                 490.0     1.252335  
1122.1 Будинки багатоквартирні масової забудови       243.0     0.800157  
1263.3 Будівлі шкіл та інших середніх навчальни...    240.0     2.100913  
1252.8 Склади універсальні                            200.0     1.034123  
1271.9 Будівлі сільськогосподарського призначен...    147.0     0.614439  
1230.3 Станції технічного обслуговування автомо...    115.0     2.394065  
1252.9 Склади та сховища інші                         107.0     0.476069  
1122.2 Будинки багатоквартирні підвищеної комфо...    105.0     0.513221

**1263.3 Будівлі шкіл та інших середніх навчальних закладів** дорожче у великих містах так само як і **1230.3 Станції технічного обслуговування автомобілів**

**1252.9 Склади та сховища інші** та **1122.2 Будинки багатоквартирні підвищеної комфортності, індивідуальні** дешевші

### [H4] Є об'єкти з підозріло коротким терміном реалізації (≤2 міс.) при високій вартості — можливі фіктивні проєкти.

Термін реаліації відсутній у багатьох об'єктах

### [H5] Ціна за м² знижується або зростає з більшою кількістю поверхів (аналіз економії масштабу).

### [H6] Об'єкти, що будуються “господарським способом”, мають системно нижчу вартість, ніж підрядні.

### [H7] У Києві/обласних центрах — вища ціна за м², ніж у райцентрах або сільській місцевості.

### [H8] Будівництво у прикордонних регіонах (Херсон, Суми, Чернігів) демонструє аномалії у вартості та темпах

### [H10] Кількість поверхів ≠ 1, але “загальна площа” дуже мала — можливе дроблення або помилки.

### [H11] Для об'єктів з ДКБС-кодом, що мають виробниче або енергетичне призначення — середня вартість суттєво відрізняється.

### [H12] Торгові приміщення (магазини) мають вартість м² близьку до житлових, але з коротшими термінами будівництва.

### [H13] Підозрілі об'єкти мають: високу вартість, короткий термін, багато “нулів” у ТЕП, нульову корисну площу або 0 поверхів.

### [H14] Найбільш аномальні об'єкти зустрічаються серед приватних замовників, а не держорганів.